<a href="https://colab.research.google.com/github/adam-bozman/hypothetical-sabbatical/blob/main/BoardExNetworks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wrds -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 73.9 MB/s eta 0:00:00


In [2]:
import wrds
import pandas as pd
import numpy as np
import datetime as dt
import wrds
import networkx as nx
import statsmodels.api as sm
#import psycopg2
import matplotlib.pyplot as plt

In [3]:
# Establish a connection
conn = wrds.Connection()

Enter your WRDS username [root]:bozmanadam
Enter your password:··········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [17]:
# Board Composition Data
bx = conn.raw_sql("""
                    select a.CompanyID, a.DirectorID, a.DirectorName, a.Seniority,
                    a.CompanyName, a.RoleName, a.DateStartRole, a.DateEndRole
                    from boardex.na_wrds_org_composition as a
                    where a.DateStartRole between '01/01/2022' and '12/31/2023'
                    """, date_cols=['date'])

In [18]:
# Convert 'datestartrole' and 'dateendrole' to datetime
bx['datestartrole'] = pd.to_datetime(bx['datestartrole'])
bx['dateendrole'] = pd.to_datetime(bx['dateendrole'])

In [22]:
# Identify the directors and executives who were serving on the SVB board at the time of the collapse
svb_directors = bx[(bx['companyname'] == 'SVB Financial Group') &
                   ((bx['dateendrole'].dt.date >= dt.date(2023, 3, 1)) |
                    (bx['dateendrole'].isna()))]

# Drop duplicates to get unique director names and IDs
svb_directors = svb_directors[['directorid', 'directorname']].drop_duplicates()

# Print the DataFrame
print(svb_directors)


       directorid       directorname
91321    344990.0        Greg Becker
91322     34775.0      Garen Staglin
91323     34868.0       Alison Davis
91592     87682.0      Eric Benhamou
91593    142073.0      Joel Friedman
91594    370982.0      Kate Mitchell
91595    489840.0        Mary Miller
91596   1037827.0           Tom King
91597   1091391.0  Jeff Maggioncalda
91598   1482935.0       Dick Daniels
91599   1812810.0   Beverly Matthews
91600   1874091.0          Busy Burr


In [23]:
# Stock Price Data
sp = conn.raw_sql("""
                    select a.permno, a.date, a.prc, a.permco
                    from crsp.dsf as a
                    where a.date between '01/01/2022' and '12/31/2023'
                    """, date_cols=['date'])

In [7]:
# Linking Data
ld = conn.raw_sql("""
                    select a.PERMCO, a.GVKEY, a.companyid,
                    a.score, a.preferred, a.duplicate
                    from wrdsapps.bdxcrspcomplink as a
                    """)

In [8]:
# First merge
merged_df = bx.merge(ld, on='companyid', how='inner')

In [9]:
# Second merge
df = merged_df.merge(sp, on='permco', how='inner')

In [11]:
df.head()
#print(len(df))

,companyid,directorid,directorname,seniority,companyname,rolename,datestartrole,dateendrole,permco,gvkey,score,preferred,duplicate,permno,date,prc
0,6.0,327069.0,Chris McCann,Executive Director,1-800-FLOWERS.COM INC,CEO,2022-04-25,NaT,16695.0,122519,2.0,1.0,0.0,87162.0,2022-01-03,23.950001
1,6.0,327069.0,Chris McCann,Executive Director,1-800-FLOWERS.COM INC,CEO,2022-04-25,NaT,16695.0,122519,2.0,1.0,0.0,87162.0,2022-01-04,24.240000
2,6.0,327069.0,Chris McCann,Executive Director,1-800-FLOWERS.COM INC,CEO,2022-04-25,NaT,16695.0,122519,2.0,1.0,0.0,87162.0,2022-01-05,24.139999
3,6.0,327069.0,Chris McCann,Executive Director,1-800-FLOWERS.COM INC,CEO,2022-04-25,NaT,16695.0,122519,2.0,1.0,0.0,87162.0,2022-01-06,24.530001
4,6.0,327069.0,Chris McCann,Executive Director,1-800-FLOWERS.COM INC,CEO,2022-04-25,NaT,16695.0,122519,2.0,1.0,0.0,87162.0,2022-01-07,24.260000


In [24]:
# Convert the director IDs to a list
svb_director_ids = svb_directors['directorid'].tolist()

# Identify the companies that have a director in common with SVB
interlocked_companies = bx[bx['directorid'].isin(svb_director_ids) &
                           (bx['datestartrole'].dt.date < dt.date(2023, 3, 1)) &
                           ((bx['dateendrole'].dt.date > dt.date(2023, 3, 1)) |
                            (bx['dateendrole'].isna()))]

In [25]:
interlocked_companies

,companyid,directorid,directorname,seniority,companyname,rolename,datestartrole,dateendrole
10933,26313.0,1874091.0,Busy Burr,Executive Director,RITE AID CORP,Interim CEO,2023-01-07,NaT
11777,27981.0,1037827.0,Tom King,Supervisory Director,SVB FINANCIAL GROUP (Silicon Valley Bancshares...,Independent Director,2022-09-13,2023-03-28
11778,27981.0,1812810.0,Beverly Matthews,Supervisory Director,SVB FINANCIAL GROUP (Silicon Valley Bancshares...,Independent Chair,2022-04-21,2023-03-28
21073,248842.0,1037827.0,Tom King,Supervisory Director,Silicon Valley Bank (Ceased Trading 03/2023),Director - SD,2022-09-13,2023-03-10
21074,248842.0,1812810.0,Beverly Matthews,Supervisory Director,Silicon Valley Bank (Ceased Trading 03/2023),Independent Chair,2022-04-21,2023-03-10
51269,1874456.0,87682.0,Eric Benhamou,Supervisory Director,Totango Inc,Director - SD,2023-01-12,NaT


In [31]:
# Get the unique company IDs for the interlocked companies
interlocked_companyids = interlocked_companies['companyid'].unique()

# Filter your larger dataset for rows where companyid is in interlocked_companyids
interlocked_data = df[df['companyid'].isin(interlocked_companyids)]

# Extract unique permno codes
interlocked_permnos = interlocked_data['permno'].unique()

# Filter the sp DataFrame for rows where permno is in interlocked_permnos
interlocked_stock_prices = sp[sp['permno'].isin(interlocked_permnos)]

In [32]:
interlocked_stock_prices

,permno,date,prc,permco
112,11786.0,2022-01-03,688.169983,9588.0
6191,46922.0,2022-01-03,15.030000,21515.0
9422,11786.0,2022-01-04,706.150024,9588.0
15502,46922.0,2022-01-04,14.480000,21515.0
18731,11786.0,2022-01-05,681.929993,9588.0
...,...,...,...,...
368719,46922.0,2022-12-28,3.270000,21515.0
371836,11786.0,2022-12-29,234.630005,9588.0
378231,46922.0,2022-12-29,3.310000,21515.0
381348,11786.0,2022-12-30,230.139999,9588.0
